In [15]:
import numpy as np
import multiprocessing
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import json
import requests
from bs4 import BeautifulSoup
import time as t

In [16]:
url = 'https://polarniczki.pl/'

In [19]:

def parsing(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        print("Połączyło się z", url)
        soup = BeautifulSoup(response.text, 'html.parser')
        titles = [text.get_text(strip=True) for text in soup.find_all('article')]
        return {"url": url, "titles": titles}
    except Exception as e:
        return {"url": url, "error": str(e)}


def main():
    start = t.time()
    base_url = 'https://polarniczki.pl/'
    subpages = [
        'idea/',
        'uczestniczki-wypraw/',
        'ksiazka/',
        'kontakt/'
    ]
    urls = [base_url + subpage for subpage in subpages]

    p_results = []
    with ProcessPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:
        p_results = [executor.submit(parsing, url) for url in urls]


    results = [r.result() for r in p_results]
    with open('parsed_data.json', 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=4)

    end = t.time()
    print("Parsowanie zakończone. Wyniki zapisano w pliku 'parsed_data.json'.")
    print("Czas wykonania [ s ]: ", round(end - start, 2))

if __name__ == "__main__":
    main()

Połączyło się z Połączyło się zhttps://polarniczki.pl/idea/ 
https://polarniczki.pl/kontakt/
Połączyło się z https://polarniczki.pl/ksiazka/Połączyło się z
 https://polarniczki.pl/uczestniczki-wypraw/
Parsowanie zakończone. Wyniki zapisano w pliku 'parsed_data.json'.
Czas wykonania [ s ]:  1.81
